<a href="https://colab.research.google.com/github/remanx2558/MIDBReviews/blob/master/RMDL_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install bert-for-tf2
!pip install sentencepiece

     |████████████████████████████████| 40kB 3.9MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.5-cp36-none-any.whl size=30317 sha256=f07782c819ba8b2b97fb1e5fddd8ffc151be5cb9895a727aabf69815c322bf6f
  Stored in directory: /root/.cache/pip/wheels/2e/70/a2/be357037dd2cbdcaeb0add1fdf083be6a600ca65ee1f68751c
  Created wheel for py-params: filename=py_params-0.9.7-cp36-none-any.whl size=7304 sha256=29128003e092485914b11dbf3158c10a869523fc22f265367b61e1602bfed84b
  Stored in directory: /root/.cache/pip/wheels/67/f5/19/b461849a50aefdf4bab47c4756596e82ee2118b8278e5a1980
  Created wheel for params-flow: filename=params_flow-0.8.2-cp36-none-any.whl size=19475 sha256=28d1b5e63662b5c4c267342159968aba6f252fa5b1c37edbea54e42273588508
  Stored in directory: /root/.cache/pip/wheels/08/c8/7f/81c86b9ff2b86e2c477e3914175be03e679e596067dc630c06
Successfully built bert-for-tf2 py-params params-flow
     |████████████████████████████████| 1.1MB 8.2MB/s 


In [2]:
try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

import tensorflow_hub as hub

from tensorflow.keras import layers
import bert

In [3]:
import pandas as pd
import numpy as np
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import os
import sys
import pickle
import itertools
import gensim
from sklearn.model_selection import train_test_split
from numpy import zeros
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from gensim.models import Word2Vec

In [4]:
# Importing dataset
reviews_df = pd.read_csv(r"/content/drive/My Drive/malia/train.csv")
reviews_df.isnull().values.any()
reviews_df.sentiment= reviews_df.sentiment.fillna(0.0).astype(int)#this will conver float into int and also manage missing values
print(reviews_df.dtypes)

text         object
sentiment     int64
dtype: object


In [22]:
def clean(text):
    '''
    '''
    text = text.lower()
    text = text.replace("ain't", "am not")
    text = text.replace("aren't", "are not")
    text = text.replace("can't", "cannot")
    text = text.replace("can't've", "cannot have")
    text = text.replace("'cause", "because")
    text = text.replace("could've", "could have")
    text = text.replace("couldn't", "could not")
    text = text.replace("couldn't've", "could not have")
    text = text.replace("should've", "should have")
    text = text.replace("should't", "should not")
    text = text.replace("should't've", "should not have")
    text = text.replace("would've", "would have")
    text = text.replace("would't", "would not")
    text = text.replace("would't've", "would not have")
    text = text.replace("didn't", "did not")
    text = text.replace("doesn't", "does not")
    text = text.replace("don't", "do not")
    text = text.replace("hadn't", "had not")
    text = text.replace("hadn't've", "had not have")
    text = text.replace("hasn't", "has not")
    text = text.replace("haven't", "have not")
    text = text.replace("haven't", "have not")
    text = text.replace("haven't", "have not")
    text = text.replace("haven't", "have not")
    text = text.replace("he'd", "he would")
    text = text.replace("haven't", "have not")
    text = text.replace("he'd've", "he would have")
    text = text.replace("'s", "")
    text = text.replace("'t", "")
    text = text.replace("'ve", "")
    text = text.replace(".", " . ")
    text = text.replace("!", " ! ")
    text = text.replace("?", " ? ")
    text = text.replace(";", " ; ")
    text = text.replace(":", " : ")
    text = text.replace(",", " , ")
    text = text.replace("´", "")
    text = text.replace("‘", "")
    text = text.replace("’", "")
    text = text.replace("“", "")
    text = text.replace("”", "")
    text = text.replace("\'", "")
    text = text.replace("\"", "")
    text = text.replace("-", "")
    text = text.replace("–", "")
    text = text.replace("—", "")
    text = text.replace("[", "")
    text = text.replace("]","")
    text = text.replace("{","")
    text = text.replace("}", "")
    text = text.replace("/", "")
    text = text.replace("|", "")
    text = text.replace("(", "")
    text = text.replace(")", "")
    text = text.replace("$", "")
    text = text.replace("+", "")
    text = text.replace("*", "")
    text = text.replace("%", "")
    text = text.replace("#", "")
    text = text.replace("\n", " \n ")
    text = text.replace("\n", "")
    text = text.replace("_", " _ ")
    text = text.replace("_", "")
    text = ''.join([i for i in text if not i.isdigit()])

    return text

texts = reviews_df['text'].values

cleaned_texts = [clean(r) for r in texts] 

reviews_df['text'] = cleaned_texts


In [23]:
# Shuffling data
reviews_df = reviews_df.sample(frac=1).reset_index(drop=True)

# Extracting all text
texts = reviews_df['text'].values
reviews_text = []

for p in zip(texts) : 
 reviews_text.append(p)



In [24]:
# Preprocessing training data


# Creating datasets
dict1 ={
    'reviews' : reviews_df['text'],
    'labels' : reviews_df['sentiment']
}
sentiment_df = pd.DataFrame.from_dict(dict1)


dict2 ={
    'reviews_text' : reviews_df['text']
}
reviews_text_df = pd.DataFrame.from_dict(dict2)

In [25]:

print(sentiment_df[:10])
print(reviews_text_df[:10])

                                             reviews  labels
0  i still have grainy  ,   late night  ,   nocab...       0
1  this film could have been a silent movie  ;   ...       0
2  this wasn the major disaster that i was expect...       1
3  it was extremely low budgetit some scenes it l...       0
4  based on a true story of how a man ahead of hi...       0
5  dolph lundgren is back  !   detention marks do...       0
6  could not believe it  !   clipped sentences  ?...       1
7  scary movie  isn as funny as its predecessors ...       1
8  stupid and just plain weird movie about some k...       1
9  i loved this movie  ,   and i am one of the ol...       0
                                        reviews_text
0  i still have grainy  ,   late night  ,   nocab...
1  this film could have been a silent movie  ;   ...
2  this wasn the major disaster that i was expect...
3  it was extremely low budgetit some scenes it l...
4  based on a true story of how a man ahead of hi...
5  dolph lu

In [26]:
def preprocess_text(sen):
    # Removing html tags
    sentence = remove_tags(sen)

    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

In [27]:
import re
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)

In [28]:
reviews = []
sentences = list(sentiment_df['reviews'])
for sen in sentences:
    reviews.append(preprocess_text(sen))

In [29]:
print(sentiment_df.columns.values)

['reviews' 'labels']


In [30]:
sentiment_df.labels.unique()

array([0, 1])

In [31]:
print(reviews[20])

robert culp they call his character doctor think he vet or something and family move to an affluent lowtax zerolawenforcement suburb lanternjawed culp and his dog are nearly killed when some local idiot neighbor kids get drunk and go cruising through his front yard at mph he presses charges which arouses the kids ire and suddenly him and his family are the victims of violent and disturbing prank campaign marilyn manson er marlyn mason rather plays his fretful boiledcelery wife who urges him not to use violence against his sneering nemeses and who really just wants to move somewhere with decent public services but the system is getting culp nowhere and he not about to leave his house because of some punk kids and their crazy rock and roll music and we all know what movie people do when the system fails but this is based on true story which makes it even better it should be noted that while the villainous hooligans do have convenient funkomatic teenage theme music that warns us when they

In [32]:
y = sentiment_df['labels']


In [33]:
BertTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)
vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertTokenizer(vocabulary_file, to_lower_case)

In [ ]:
tokenizer.convert_tokens_to_ids(tokenizer.tokenize(reviews[10]))

In [35]:
def tokenize_reviews(reviews):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(reviews))

In [36]:
tokenized_reviews = [tokenize_reviews(review) for review in reviews]

In [38]:
print(tokenized_reviews[10])


[1045, 2001, 2061, 9364, 1999, 2023, 3185, 2079, 2025, 2113, 2172, 2055, 1996, 2995, 2466, 2061, 2001, 9461, 2000, 2156, 2009, 2377, 2041, 2006, 2143, 1998, 16957, 2870, 2055, 2210, 14704, 1997, 2381, 2007, 2107, 3928, 2995, 2466, 1998, 2307, 5889, 2009, 2790, 2066, 2469, 10273, 5257, 2092, 4873, 1996, 9000, 3478, 2068, 2009, 2001, 2061, 7932, 2003, 2023, 3185, 2055, 2010, 5593, 2010, 2293, 2166, 2010, 2219, 11980, 2010, 4092, 3754, 2010, 6896, 2005, 1996, 9776, 10047, 2469, 2045, 2003, 2126, 2000, 13265, 2035, 1997, 2216, 2477, 2046, 2204, 2466, 2021, 2023, 3185, 2347, 2009, 2001, 2187, 3147, 3666, 3494, 2008, 2020, 4895, 25421, 3468, 2025, 2138, 1997, 2037, 13597, 2021, 2138, 1997, 2037, 12857, 2060, 2235, 6218, 2229, 1996, 3082, 11774, 2098, 6050, 2009, 1996, 26232, 2057, 2131, 2009, 2525, 2065, 2002, 2107, 13352, 2140, 2270, 5882, 2339, 4694, 2057, 5845, 2000, 2062, 2084, 1055, 3490, 29519, 2182, 1998, 2045, 1998, 2130, 2059, 3262, 1999, 18318, 13923]


In [40]:
reviews_with_len = [[review, y[i]]
                 for i, review in enumerate(tokenized_reviews)]

In [41]:
import random
random.shuffle(reviews_with_len)

In [42]:
print(reviews_with_len[:2])

[[[1996, 2518, 2008, 2428, 4152, 2033, 2055, 2023, 3185, 2008, 2003, 1996, 2518, 2055, 2023, 3185, 2008, 3084, 2033, 8186, 5665, 2003, 2008, 2619, 2941, 3825, 2000, 2031, 2009, 2081, 2045, 2003, 7078, 2053, 3800, 2005, 1996, 4598, 1997, 2023, 3185, 2009, 2003, 2025, 17115, 2009, 2003, 2025, 2245, 4013, 22776, 2009, 2003, 2025, 14036, 2009, 2003, 2025, 2204, 2009, 2003, 5777, 17357, 2081, 1997, 22330, 7088, 3207, 1996, 4966, 2553, 22963, 2009, 2000, 10503, 6965, 4763, 2757, 1998, 2261, 2060, 11519, 5691, 2437, 1996, 12127, 15561, 16124, 2135, 5793, 2009, 2003, 2498, 2066, 2151, 2060, 3185, 2412, 2081, 2009, 2003, 2521, 2521, 4788, 1996, 4447, 1997, 2019, 5186, 16880, 4566, 2017, 2097, 2196, 5293, 2024, 1996, 5662, 1997, 2028, 10228, 1997, 27136, 2075, 3586, 7510, 2015, 3531, 2079, 2025, 2412, 5949, 2115, 2051, 3666, 2023, 3538, 1997, 11669, 2138, 2009, 2089, 2191, 2017, 25403, 1996, 2158, 2040, 2626, 2023, 3185, 2323, 2022, 8342, 2125, 1996, 3213, 9054, 5779, 2862, 1998, 2196, 3039, 200

In [43]:
reviews_with_len.sort(key=lambda x: x[2])

IndexError: ignored

In [45]:
sorted_reviews_labels = [(review_lab[0], review_lab[1]) for review_lab in reviews_with_len]


In [46]:
processed_dataset = tf.data.Dataset.from_generator(lambda: sorted_reviews_labels, output_types=(tf.int32, tf.int32))

In [47]:
BATCH_SIZE = 32
batched_dataset = processed_dataset.padded_batch(BATCH_SIZE, padded_shapes=((None, ), ()))

In [48]:


next(iter(batched_dataset))



(<tf.Tensor: shape=(32, 1015), dtype=int32, numpy=
 array([[ 1996,  2518,  2008, ...,     0,     0,     0],
        [ 2364,  4323,  1999, ...,     0,     0,     0],
        [ 2009,  3504,  2066, ...,     0,     0,     0],
        ...,
        [ 2250, 12155, 10270, ...,     0,     0,     0],
        [ 1999,  3290,  2103, ...,     0,     0,     0],
        [ 2750,  2108,  3185, ...,     0,     0,     0]], dtype=int32)>,
 <tf.Tensor: shape=(32,), dtype=int32, numpy=
 array([1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1,
        0, 0, 1, 1, 1, 1, 0, 0, 0, 1], dtype=int32)>)

In [49]:

import math
TOTAL_BATCHES = math.ceil(len(sorted_reviews_labels) / BATCH_SIZE)
TEST_BATCHES = TOTAL_BATCHES // 10
batched_dataset.shuffle(TOTAL_BATCHES)
test_data = batched_dataset.take(TEST_BATCHES)
train_data = batched_dataset.skip(TEST_BATCHES)



In [50]:
import tensorflow as tf
import keras

In [51]:
class TEXT_MODEL(tf.keras.Model):
    
    def __init__(self,
                 vocabulary_size,
                 embedding_dimensions=128,
                 cnn_filters=50,
                 dnn_units=512,
                 model_output_classes=2,
                 dropout_rate=0.1,
                 training=False,
                 name="text_model"):
        super(TEXT_MODEL, self).__init__(name=name)
        
        self.embedding = layers.Embedding(vocabulary_size,
                                          embedding_dimensions)
        #self.rnn_layer = layers.Bidirectional(self.rnn_layer.layers.LSTM(32)),
        self.cnn_layer1 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=2,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer2 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=3,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer3 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=4,
                                        padding="valid",
                                        activation="relu")
        self.pool = layers.GlobalMaxPool1D()
        
        self.dense_1 = layers.Dense(units=dnn_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        if model_output_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=model_output_classes,
                                           activation="softmax")
    
    def call(self, inputs, training):
        l = self.embedding(inputs)
        l_1 = self.cnn_layer1(l) 
        l_1 = self.pool(l_1) 
        l_2 = self.cnn_layer2(l) 
        l_2 = self.pool(l_2)
        l_3 = self.cnn_layer3(l)
        l_3 = self.pool(l_3) 
        concatenated = tf.concat([l_1, l_2, l_3,], axis=-1) # (batch_size, 4 * rmdl_filters)
        concatenated = self.dense_1(concatenated)
        concatenated = self.dropout(concatenated, training)
        model_output = self.last_dense(concatenated)
        
        return model_output

In [52]:
VOCAB_LENGTH = len(tokenizer.vocab)
EMB_DIM = 200
CNN_FILTERS = 100
DNN_UNITS = 256
OUTPUT_CLASSES = 2

DROPOUT_RATE = 0.2

NB_EPOCHS = 5

In [53]:


text_model = TEXT_MODEL(vocabulary_size=VOCAB_LENGTH,
                        embedding_dimensions=EMB_DIM,
                        cnn_filters=CNN_FILTERS,
                        dnn_units=DNN_UNITS,
                        model_output_classes=OUTPUT_CLASSES,
                        dropout_rate=DROPOUT_RATE)



In [54]:


if OUTPUT_CLASSES == 2:
    text_model.compile(loss="binary_crossentropy",
                       optimizer="adam",
                       metrics=["accuracy"])
else:
    text_model.compile(loss="sparse_categorical_crossentropy",
                       optimizer="adam",
                       metrics=["sparse_categorical_accuracy"])



In [57]:
text_model.fit(train_data, epochs=NB_EPOCHS,validation_data=test_data)

Epoch 1/5
704/704 [==============================] - 83s 118ms/step - loss: 0.3534 - accuracy: 0.8354 - val_loss: 0.2596 - val_accuracy: 0.8934
Epoch 2/5
704/704 [==============================] - 64s 90ms/step - loss: 0.1248 - accuracy: 0.9542 - val_loss: 0.3261 - val_accuracy: 0.8846
Epoch 3/5
704/704 [==============================] - 64s 90ms/step - loss: 0.0540 - accuracy: 0.9807 - val_loss: 0.4151 - val_accuracy: 0.8834
Epoch 4/5
704/704 [==============================] - 64s 91ms/step - loss: 0.0472 - accuracy: 0.9821 - val_loss: 0.3733 - val_accuracy: 0.9054
Epoch 5/5
704/704 [==============================] - 63s 90ms/step - loss: 0.0130 - accuracy: 0.9959 - val_loss: 0.4748 - val_accuracy: 0.8906


In [58]:
loss, accuracy = text_model.evaluate(test_data)
print('Test Loss: {}'.format(loss))
print('Test Accuracy: {}'.format(accuracy))


78/78 [==============================] - 1s 16ms/step - loss: 0.4748 - accuracy: 0.8906
Test Loss: 0.4747582972049713
Test Accuracy: 0.890625
